In [8]:
import sys
import os
import pandas as pd

In [9]:
sys.path.append(os.path.abspath(".."))

from src.config import UNIVERSE

UNIVERSE


['XLK',
 'XLC',
 'XLY',
 'XLP',
 'XLF',
 'XLV',
 'XLE',
 'XLI',
 'XLB',
 'XLRE',
 'XLU',
 'AAPL',
 'MSFT',
 'NVDA',
 'GOOGL',
 'META',
 'NFLX',
 'AMZN',
 'TSLA',
 'MCD',
 'PG',
 'KO',
 'PEP',
 'JPM',
 'BAC',
 'V',
 'JNJ',
 'PFE',
 'UNH',
 'XOM',
 'CVX',
 'COP',
 'BA',
 'CAT',
 'GE',
 'LIN',
 'SHW',
 'FCX',
 'AMT',
 'PLD',
 'EQIX',
 'NEE',
 'DUK',
 'SO']

In [10]:
import glob, os

files = glob.glob("../data/raw/D1/*.US_D1.csv")

available = {
    os.path.basename(f).split(".")[0]   # like "AAPL"
    for f in files
}

valid_universe = [t for t in UNIVERSE if t in available]
missing = [t for t in UNIVERSE if t not in available]

len(valid_universe), missing[:20]


(33,
 ['XLK',
  'XLC',
  'XLY',
  'XLP',
  'XLF',
  'XLV',
  'XLE',
  'XLI',
  'XLB',
  'XLRE',
  'XLU'])

In [11]:
dfs = []

for ticker in valid_universe:
    path = f"../data/raw/D1/{ticker}.US_D1.csv"
    df_tmp = pd.read_csv(path)
    df_tmp["ticker"] = ticker
    dfs.append(df_tmp)

df = pd.concat(dfs, ignore_index=True)

df["datetime"] = pd.to_datetime(df["datetime"])
df = df.sort_values(["ticker", "datetime"]).reset_index(drop=True)

df.shape


(200892, 7)

In [12]:
df = df[df["datetime"] >= "2010-01-04"].copy()

df.shape

(118620, 7)

In [13]:
df.head()

,datetime,open,high,low,close,volume,ticker
3019,2010-01-04,7.62,7.65,7.58,7.64,116694802,AAPL
3020,2010-01-05,7.65,7.69,7.61,7.65,136014592,AAPL
3021,2010-01-06,7.65,7.68,7.52,7.53,133300727,AAPL
3022,2010-01-07,7.56,7.57,7.46,7.52,113809059,AAPL
3023,2010-01-08,7.51,7.57,7.46,7.56,104221936,AAPL


In [14]:
df.columns

Index(['datetime', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='str')

In [15]:
df.duplicated(subset=["ticker", "datetime"]).sum()

np.int64(0)

In [16]:
df.isna().sum()

datetime    0
open        0
high        0
low         0
close       0
volume      0
ticker      0
dtype: int64

In [17]:
df.groupby("ticker")["datetime"].is_monotonic_increasing.all()

np.True_

In [18]:
df.groupby("ticker").size().describe()

count      33.000000
mean     3594.545455
std       106.534939
min      3021.000000
25%      3619.000000
50%      3620.000000
75%      3620.000000
max      3620.000000
dtype: float64

In [19]:
from src.features import add_features

df = add_features(df)

df.shape

(116541, 10)

In [20]:
from src.labels_long import add_long_score

df = add_long_score(df)

df.shape

(114462, 14)

## Walk-forward Split

In [21]:
df["ret_long_norm"].describe()

count    114462.000000
mean          1.729788
std          10.435223
min         -75.973971
25%          -2.379906
50%           1.634154
75%           5.698800
max         425.753798
Name: ret_long_norm, dtype: float64

In [23]:
dates = pd.Index(df["datetime"].unique()).sort_values()

min_train_days = 756   # ~3 years
test_days = 126        # ~6 months
step_days = 63         # ~3 months

splits = []
for i in range(min_train_days, len(dates) - test_days, step_days):
    train_end = dates[i-1]
    test_start = dates[i]
    test_end = dates[i + test_days - 1]
    splits.append((train_end, test_start, test_end))

len(splits), splits[0], splits[-1]


(42,
 (Timestamp('2013-04-05 00:00:00'),
  Timestamp('2013-04-08 00:00:00'),
  Timestamp('2013-10-03 00:00:00')),
 (Timestamp('2023-05-25 00:00:00'),
  Timestamp('2023-05-26 00:00:00'),
  Timestamp('2023-11-21 00:00:00')))

In [24]:
train_end, test_start, test_end = splits[0]
train = df[df["datetime"] <= train_end]
test  = df[(df["datetime"] >= test_start) & (df["datetime"] <= test_end)]

train.shape, test.shape

((24226, 14), (4158, 14))

In [26]:
import numpy as np

# thresholds ONLY from train
lower = train["ret_long_norm"].quantile(0.30)
upper = train["ret_long_norm"].quantile(0.70)

# apply to both
def make_label(s, lo, hi):
    return np.where(s < lo, "Down", np.where(s > hi, "Up", "Neutral"))

train["long_label"] = make_label(train["ret_long_norm"], lower, upper)
test["long_label"]  = make_label(test["ret_long_norm"],  lower, upper)

train["long_label"].value_counts(), test["long_label"].value_counts()

(long_label
 Neutral    9690
 Up         7268
 Down       7268
 Name: count, dtype: int64,
 long_label
 Neutral    1585
 Up         1340
 Down       1233
 Name: count, dtype: int64)